# Use Pandas to select pairs from ASF inventory

In [3]:
# if dinosar library not in base environment uncomment below (run just once)
!pip install --no-cache git+https://github.com/scottyhq/dinosar.git@master

  Cloning https://github.com/scottyhq/dinosar.git (to revision master) to /tmp/pip-req-build-qqu8rffq
  Running command git clone -q https://github.com/scottyhq/dinosar.git /tmp/pip-req-build-qqu8rffq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
ERROR: Package 'dinosar' requires a different Python: 3.7.6 not in '>=3.8,<4.0'


In [2]:
import subprocess
import os
import dinosar
import geopandas as gpd
import getpass

## update ASF inventory

In [3]:
def refresh_inventory(snwe, orbit=76):
    dinosar.archive.asf.query_asf(snwe, 'SA', orbit=relOrbit)
    dinosar.archive.asf.query_asf(snwe, 'SB', orbit=relOrbit)
    gf = dinosar.archive.asf.merge_inventories('query_SA.json', 'query_SB.json')
    dinosar.archive.asf.save_inventory(gf)

In [4]:
relOrbit = 76
snwe = [-23.0, -21.5, -68, -66.5]
refresh_inventory(snwe, relOrbit)

Querying ASF Vertex for SA...
https://api.daac.asf.alaska.edu/services/search/param?intersectsWith=POLYGON+%28%28-66.5+-23%2C+-66.5+-21.5%2C+-68+-21.5%2C+-68+-23%2C+-66.5+-23%29%29&platform=SA&processingLevel=SLC&beamMode=IW&output=json&relativeOrbit=76
Querying ASF Vertex for SB...
https://api.daac.asf.alaska.edu/services/search/param?intersectsWith=POLYGON+%28%28-66.5+-23%2C+-66.5+-21.5%2C+-68+-21.5%2C+-68+-23%2C+-66.5+-23%29%29&platform=SB&processingLevel=SLC&beamMode=IW&output=json&relativeOrbit=76
Merging S1A and S1B inventories
Saved inventory:  query.geojson


In [5]:
gf = dinosar.archive.asf.load_inventory('query.geojson')

In [6]:
# Useful for exploring data
#gf.head()
#print(gf.iloc[0].to_string())
# See also https://www.earthdatascience.org/courses/use-data-open-source-python/use-time-series-data-in-python/date-time-types-in-pandas-python/subset-time-series-data-python/

In [7]:
# don't care about geometry and other metadata 
# generally two frames intersect AOI per date:
df = gf.loc[:,['frameNumber','dateStamp','platform']].sort_values(by='dateStamp')
df.head()

,frameNumber,dateStamp,platform
240,1105,2014-11-03,Sentinel-1A
239,1110,2014-11-03,Sentinel-1A
237,1110,2014-11-27,Sentinel-1A
238,1105,2014-11-27,Sentinel-1A
236,1105,2014-12-21,Sentinel-1A


In [8]:
# Take only unique dates and set DateTimeIndex
df  = df.drop_duplicates('dateStamp').set_index('dateStamp', drop=False)
df.head()

,frameNumber,dateStamp,platform
dateStamp,,,
2014-11-03,1105,2014-11-03,Sentinel-1A
2014-11-27,1110,2014-11-27,Sentinel-1A
2014-12-21,1105,2014-12-21,Sentinel-1A
2015-01-14,1104,2015-01-14,Sentinel-1A
2015-02-07,1105,2015-02-07,Sentinel-1A


In [9]:
# First date in june of every year
df[df.index.month == 6].resample('Y').first().sort_index(ascending=False)

,frameNumber,dateStamp,platform
dateStamp,,,
2019-12-31,1105,2019-06-04,Sentinel-1B
2018-12-31,1101,2018-06-03,Sentinel-1A
2017-12-31,1101,2017-06-02,Sentinel-1B
2016-12-31,1106,2016-06-13,Sentinel-1A
2015-12-31,1105,2015-06-19,Sentinel-1A


## pairs between june acquistions every year

In [11]:
def dates_to_pairs(dates):
    pairs = []
    for i in range(len(dates)-1):
        master = dates[i]
        second = dates[i+1]
        intdir = 'int-{0}-{1}'.format(master, second)
        pairs.append(intdir)
    
    return pairs

In [12]:
def one_per_june(df):
    # Single date per month
    tmp = df[df.index.month == 6].resample('Y').first()
    tmp1 = tmp.sort_index(ascending=False)
    dates = tmp1.dateStamp.apply(lambda x: x.strftime('%Y%m%d')).to_list()
    pairs = dates_to_pairs(dates)
    
    return pairs

In [13]:
pairs = one_per_june(df)
print(f'{len(pairs)} Interferograms to generate:')
pairs

4 Interferograms to generate:


['int-20190604-20180603',
 'int-20180603-20170602',
 'int-20170602-20160613',
 'int-20160613-20150619']

## one acquistion from every month

In [14]:
def one_per_month(df):
    # Single date per month
    tmp = df.resample('M').first()
    tmp1 = tmp.sort_index(ascending=False)
    dates = tmp1.dateStamp.apply(lambda x: x.strftime('%Y%m%d')).to_list()
    pairs = dates_to_pairs(dates)
    
    return pairs

In [15]:
pairs = one_per_month(df)
print(f'{len(pairs)} Interferograms to generate:')
pairs

66 Interferograms to generate:


['int-20200505-20200405',
 'int-20200405-20200306',
 'int-20200306-20200205',
 'int-20200205-20200106',
 'int-20200106-20191201',
 'int-20191201-20191101',
 'int-20191101-20191002',
 'int-20191002-20190902',
 'int-20190902-20190803',
 'int-20190803-20190704',
 'int-20190704-20190604',
 'int-20190604-20190505',
 'int-20190505-20190405',
 'int-20190405-20190306',
 'int-20190306-20190210',
 'int-20190210-20190105',
 'int-20190105-20181206',
 'int-20181206-20181106',
 'int-20181106-20181001',
 'int-20181001-20180907',
 'int-20180907-20180802',
 'int-20180802-20180703',
 'int-20180703-20180603',
 'int-20180603-20180504',
 'int-20180504-20180404',
 'int-20180404-20180305',
 'int-20180305-20180209',
 'int-20180209-20180104',
 'int-20180104-20171211',
 'int-20171211-20171105',
 'int-20171105-20171012',
 'int-20171012-20170906',
 'int-20170906-20170801',
 'int-20170801-20170708',
 'int-20170708-20170602',
 'int-20170602-20170509',
 'int-20170509-20170415',
 'int-20170415-20170304',
 'int-201703